# Consistency Check Agent 테스트

일관성 검사 에이전트 (Level 2) 테스트 노트북

In [1]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

Project root: c:\jungle\weapon\sto-link-AI-backend


In [2]:
def create_base_state():
    return {"content": "테스트 스토리", "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/api/callback",
            "extracted_characters": [], "extracted_events": [],
            "relationship_graph": {}, "consistency_report": {},
            "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

In [3]:
from app.agents.analysis.consistency import consistency_check_node

# 정상 케이스 테스트
async def test_consistency_normal():
    print("✅ Consistency Check (정상) 테스트...")
    state = create_base_state()
    state["extracted_characters"] = [
        {"name": "서진", "role": "protagonist", "traits": ["brave", "skilled"], "status": "alive"},
        {"name": "하나", "role": "supporting", "traits": ["wise", "calm"], "status": "alive"}
    ]
    state["extracted_events"] = [
        {"event_id": "E001", "description": "숲에서 만남", "participants": ["서진", "하나"]}
    ]
    return await consistency_check_node(state)

result = run_async(test_consistency_normal())
print(json.dumps(result, ensure_ascii=False, indent=2))

✅ Consistency Check (정상) 테스트...
[CONSISTENCY] LLM error: Extra data: line 7 column 1 (char 65)
[CONSISTENCY] Score: 100, HIGH: 0, MEDIUM: 0, Reextract: False
{
  "consistency_report": {
    "overall_score": 100,
    "conflicts": [],
    "warnings": [],
    "requires_reextraction": false
  },
  "messages": [
    {
      "role": "consistency_agent",
      "content": "Score: 100, Conflicts: 0 (HIGH: 0)"
    }
  ]
}


In [4]:
# 충돌 케이스 테스트 (모순된 traits)
async def test_consistency_conflict():
    print("⚠️ Consistency Check (충돌) 테스트...")
    state = create_base_state()
    state["extracted_characters"] = [
        {"name": "모순캐릭터", "role": "protagonist", "traits": ["coward", "brave", "timid"], "status": "alive"},
        {"name": "또다른모순", "role": "supporting", "traits": ["weak", "strong"], "status": "alive"}
    ]
    state["extracted_events"] = [
        {"event_id": "E001", "description": "모순캐릭터가 용감하게 싸웠다", "participants": ["모순캐릭터"]}
    ]
    return await consistency_check_node(state)

conflict_result = run_async(test_consistency_conflict())
print(json.dumps(conflict_result, ensure_ascii=False, indent=2))

⚠️ Consistency Check (충돌) 테스트...
[CONSISTENCY] Programmatic conflict added: Character '모순캐릭터' has contradictory traits: 'coward' and 'br...
[CONSISTENCY] Programmatic conflict added: Character '모순캐릭터' has contradictory traits: 'timid' and 'bra...
[CONSISTENCY] Programmatic conflict added: Character '또다른모순' has contradictory traits: 'weak' and 'stro...
[CONSISTENCY] Score: 0, HIGH: 5, MEDIUM: 1, Reextract: True
{
  "consistency_report": {
    "overall_score": 0,
    "conflicts": [
      {
        "type": "CHARACTER_TRAIT_CONFLICT",
        "description": "Character 모순캐릭터 has conflicting traits: coward, brave, timid",
        "severity": "HIGH",
        "affected_elements": [
          "모순캐릭터"
        ]
      },
      {
        "type": "CHARACTER_TRAIT_CONFLICT",
        "description": "Character 또다른모순 has conflicting traits: weak, strong",
        "severity": "HIGH",
        "affected_elements": [
          "또다른모순"
        ]
      },
      {
        "type": "EVENT_CONTRADICTION",
      

In [5]:
# 프로그래매틱 충돌 감지 테스트
from app.agents.analysis.consistency import detect_trait_contradictions

test_characters = [
    {"name": "테스트1", "traits": ["coward", "brave"]},
    {"name": "테스트2", "traits": ["honest", "liar"]},
    {"name": "정상", "traits": ["brave", "kind"]}
]

conflicts = detect_trait_contradictions(test_characters)
print("감지된 충돌:")
print(json.dumps(conflicts, ensure_ascii=False, indent=2))

감지된 충돌:
[
  {
    "type": "CHARACTER_TRAIT_CONFLICT",
    "description": "Character '테스트1' has contradictory traits: 'coward' and 'brave' cannot both be true.",
    "severity": "HIGH",
    "affected_elements": [
      "테스트1"
    ],
    "programmatic": true
  }
]
